In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

# Load the question generation pipeline
question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# Sample passage
passage = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals.
Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.
"""

# Prepare the input text for question generation
input_text = f"generate questions: {passage}"

# Generate questions using beam search
questions = question_generator(input_text, max_length=512, num_beams=5, num_return_sequences=3)

# Display the generated questions
for i, q in enumerate(questions):
    print(f"Q{i+1}: {q['generated_text']}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Q1: What does the term "intelligent agent" refer to?
Q2: What does the term "intelligent agent" mean?
Q3: What does the term "intelligent agents" refer to?


In [ ]:
!pip install haystack

In [ ]:
!pip install nodes

  Using cached Nodes-1.2.tar.gz (190 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
pip install --upgrade farm-haystack

In [ ]:
!pip install farm-haystack

In [ ]:
from haystack.nodes import QuestionGenerator
from haystack.utils import clean_wiki_text

# Initialize the question generator
question_generator = QuestionGenerator(model_name_or_path="valhalla/t5-base-qg-hl")

# Sample passage
passage = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals.
Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.
"""

# Clean the text (if necessary, based on the source of the text)
clean_passage = clean_wiki_text(passage)

# Generate questions
questions = question_generator.generate(clean_passage)

# Inspect the format of generated questions
print(questions)

# Display the generated questions
for i, q in enumerate(questions):
    print(f"Q{i+1}: {q}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['What is the definition of intelligent agents?']
Q1: What is the definition of intelligent agents?


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the model and tokenizer
model_name = "valhalla/t5-base-qg-hl"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_questions(passage):
    # Prepare the input text
    input_text = "generate questions: " + passage
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate questions
    outputs = model.generate(
        input_ids=input_ids,
        max_length=256,
        num_return_sequences=5,
        num_beams=5
    )

    # Decode the generated questions
    questions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return questions

# Sample passage
passage = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals.
Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.
"""

# Generate questions
questions = generate_questions(passage)

# Display the generated questions
for i, q in enumerate(questions):
    print(f"Q{i+1}: {q}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Q1: What does the term "intelligent agent" refer to?
Q2: What does the term "intelligent agent" mean?
Q3: What does the term "intelligent agents" refer to?
Q4: What does the term "intelligent agents" mean?
Q5: What is the definition of intelligent agents?


In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")

context = r"""
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals.
Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that
maximize its chance of successfully achieving its goals.
"""

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
result = question_answerer(question="What does the term 'intelligent agent' refer to?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

result = question_answerer(question="What is Artificial intelligence?", context=context)
print(
    f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}"
)

Answer: 'any device that perceives its environment', score: 0.23, start: 221, end: 262
Answer: 'intelligence demonstrated by machines', score: 0.906, start: 33, end: 70


In [ ]:
# prompt: getting error

!pip install --upgrade transformers


In [ ]:
!pip install transformers
!pip install datasets
!pip install faiss-cpu  # or faiss-gpu if you have a GPU

In [ ]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration

# Initialize RAG components
tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
retriever = RagRetriever.from_pretrained("facebook/rag-token-base")
model = RagTokenForGeneration.from_pretrained("facebook/rag-token-base")

# Sample passage
passage = """
Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals.
Leading AI textbooks define the field as the study of "intelligent agents": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals.
"""

# Retrieve relevant documents from the passage
input_ids = tokenizer("question: " + passage, return_tensors="pt")["input_ids"]
retrieved_docs = retriever(input_ids)

# Generate questions from the retrieved documents
generated_questions = model.generate(input_ids=retrieved_docs["input_ids"], num_questions=3)

# Decode and print the generated questions
for i, question in enumerate(generated_questions):
    decoded_question = tokenizer.decode(question, skip_special_tokens=True)
    print(f"Question {i+1}: {decoded_question}")



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

ImportError: 
RagRetriever requires the 🤗 Datasets library but it was not found in your environment. You can install it with:
```
pip install datasets
```
In a notebook or a colab, you can install it by executing a cell with
```
!pip install datasets
```
then restarting your kernel.

Note that if you have a local folder named `datasets` or a local python file named `datasets.py` in your current
working directory, python may try to import this instead of the 🤗 Datasets library. You should rename this folder or
that python file if that's the case. Please note that you may need to restart your runtime after installation.

RagRetriever requires the faiss library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/facebookresearch/faiss/blob/master/INSTALL.md and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.
